## Bernoulli Factories MCMC

If sampling from the MCMC algorithms like MH algorithm, we need to calculate the acceptance probability and in many cases the functional form is intracable or very difficult to calculate Accept-reject ratio.<br>We can use Bernoulli Factories instead of normal accept reject and calculate the ratio without even calculating it.The two-coin algorithm of *Goncalves* on each iteration of MCMC implement Bernoulli Factory to make choice. Hence using some acceptance for which Bernoulli Factory are easy to implement can help.

### Barker's Method and the two-coin algorithm.
Similiar to MH-Algo, Barker's Method uses a proposal q(x,y). But, instead comparing the ratio $\alpha_B$ with $U \sim U[0,1]$, we sample from bernoulli with probability of Barker's Ratio.Hence, the modified Algorithm is written as:
1. Draw y from proposal
2. Now sample A from Bernoulli of p=$\alpha_B$
3. If A=1 then update the next state
4. Else state remains same.

Now, implementing the Bernoulli factory for the Algorithm. Suppose, $\pi(X)q(X,Y)=c_xp_x$, where $p_x$ can be assumed of a probability and $c_x$ be the upper bound on the probability, similiarly for state y. So, modified Two-coin algo produce output with probability as $\alpha_B$.<br>
Two-Coin Algorithm is written below:
1. Sample from C1 $\sim$ Bern($\frac{c_y}{c_x+c_y}$)
2. If C1==1 do the following
    1. Draw C2 = Bern($p_y$)
    2. If C2==1 return 1
    3. Else Goto step 1
3. If C1==0 do the following
    1. Draw C2 = Bern($p_x$)
    2. If C2==1 return 0
    3. Else Goto step 1

The Efficiency of Agorithm Depends on the bound of $C_x$ and $C_y$

### PortKey's Barker Method
The Inefficiency of above method is due to Acceptance ratio, so finding a efficient ratio can increase the efficiency of Two Coin Method.<br>
Consider, $\alpha(x, y) = \frac{\pi(y)q(y,x)}{\pi(x)q(x,y)+\pi(y)q(y,x)+d(x,y)}$ as member of Acceptance Probability of the family.<br>
Below two properties are required to use it as acceptance probability:-
1. $\alpha$ is $\pi-reversible$ if d(x,y)=d(y,x)
    So, we can use a d(x,y) = $\frac{1-\beta}{\beta}(c_x+c_y)$ and modify the above algorithm to get the *PortKey* Barker Method.The $\beta$ should be close to 1 to reduce the d(x,y) error.
2. The Following modified Algorithm will give output 1 with probability $\alpha_\beta(x,y)$

Modification's in Two-Key Algorithm:-
1. Initially sample S from Bernoulli($\beta$).
2. If S==0 return 0
3. Else Do the Two Coin Algorithm.

The average number of Loop of Both the above Algorithm have the ratio which shows that the new Portkey's two coin method is efficient/comparable than the original two coin method.<br>
The computational Efficiency come here at cost of Statistical Error.
<br>
For 0 < β ≤ 1, α(β) (x, y) ≤ β$α_B$(x, y)<br> var(g, $P_B$) ≤ β var(g, P(β)) + (β − 1)$Var_\pi$(g).

### Flipped PortKey Two Coin Algorithm
Sometimes, finding upper bound of $\pi(x)q(x,y)$ so we can use the lower bound in that case. Such that<br>
$\pi(x)^{-1}q(x,y)^{-1} = \tilde{c_x}\tilde{p}(x)$ and<br>
$\tilde{d}$(x, y) = $\frac{1-\beta}{\beta}\frac{\tilde{c_x}+\tilde{c_y}}{\tilde{c_xp_x}+\tilde{c_yp_y}}$<br>
Then using similar to Portkey's Algorithm we can apply Flipped Portkey's Agorithm

# Q2

In [1]:
using Distributions

In [2]:
a=2
b=5
function cθ(x, k)
    return (k/(exp(1)*x))
end
function pθ(x, k)                     
    λ = rand(Gamma(a,b))         
    return pdf(Weibull(k,λ), x)/cθ(x, k)
end

pθ (generic function with 1 method)

In [142]:
pθ(1,2)

0.20298070934236268

In [3]:
function portkey(β, x, y, cx, cy, k)
    flag=0
    run_count = 0;
    ratio = (cy)/(cx+cy)
    while(flag==0)
        run_count = run_count + 1
        S = rand(Bernoulli(β))
        if S==false
            flag=1
            return [x run_count]
        else
            C1 = rand(Bernoulli(ratio))
            if C1==true
                C2 = rand(Bernoulli(pθ(y, k)))
                if C2==true
                    flag=1
                    return [y run_count]
                end
            else
                C2 = rand(Bernoulli(pθ(x, k)))
                if C2==true
                    flag=1
                    return [x run_count]
                end
            end
        end
    end
end

portkey (generic function with 1 method)

In [4]:
k = 10
function mcmc(β)
    var = 0.001
    iterations = 1000000
    observe = [1.0]
    run_time = []
    accept = 0
    for i=1:iterations
        y = rand(Normal(observe[i], sqrt(var)))
        if(y<0)
            push!(observe, observe[i])
            continue
        end
        alpha = portkey(β, observe[i], y, cθ(observe[i], k), cθ(y, k), k)
        push!(observe, alpha[1])
        if(observe[i+1]!=observe[i])
            accept = accept+1
        end
        push!(run_time, alpha[2])
    end
    println(accept/iterations)
    return observe, run_time
end

mcmc (generic function with 1 method)

In [5]:

list1, number = mcmc(1);
list2, number = mcmc(0.99);
list3, number = mcmc(0.98);
list4, number = mcmc(0.97);

0.499501
0.434132
0.37058
0.345878


In [7]:
using Plots
using StatsPlots
using StatsBase

In [ ]:
p1 = plot(list1, title="β=1")
p2 = plot(list2, title="β=0.99")
p3 = plot(list3, title="β=0.98")
p4 = plot(list4, title="β=0.97")

In [ ]:
plot(p1, p2, p3, p4, layout=(2,2), legend=false)